In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['.DS_Store', 'test.csv', 'train.csv']


In [2]:
import pandas as pd
import string, re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import confusion_matrix
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Embedding, Dropout
from keras import metrics
from keras.utils import to_categorical

Using TensorFlow backend.


In [4]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression

In [5]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [8]:
train_raw_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
train_df = train_raw_df
train_df = train_raw_df.sample(frac=0.1)
train_df = train_df.append(train_raw_df[train_raw_df['target'] == 1])
train_n = len(train_df)
test_n = len(test_df)
print("Train shape : ",train_df.shape)
print("Test shape : ",test_df.shape)

Train shape :  (211422, 3)
Test shape :  (56370, 2)


In [9]:
train_df.groupby('target').count()

,qid,question_text
target,,
0,122513,122513
1,88909,88909


In [10]:
train_df.head(1)
test_df.iloc[594,:][1]

'Has anyone gotten banned on Quora for being under 13?'

In [11]:
lemmatizer = WordNetLemmatizer()
stopset = set(stopwords.words('english'))

In [12]:
def process(txt):
    tokens = []
    for token in wordpunct_tokenize(txt):
        if token.isdigit():
            continue
        if all(char in string.punctuation for char in token):
            continue
        
        token = token.lower()
        token = token.strip()  # Strip whitespace and other punctuations
        token = token.strip('_')  # remove _ if any
        token = token.strip('*')
        if token in stopset:
            continue
        tokens.append(token)
        lemmatizer.lemmatize(token)
    return tokens

In [13]:
train_df['p_txt'] = train_df['question_text'].apply(process)
test_df['p_txt'] = test_df['question_text'].apply(process)

In [14]:
train_df.head()

,qid,question_text,target,p_txt
1302852,ff5ce163099aa6075efc,What are some examples of well-written economi...,0,"[examples, well, written, economic, papers]"
898515,b00b8adb04ab754554ca,What are some tips for starting a new school y...,0,"[tips, starting, new, school, junior, year]"
1182181,e7aa94d5d64f6cb52199,What is NBT (National Book Trust)?,0,"[nbt, national, book, trust]"
627266,7ad84f5c5aa7416f9a27,Are there any existing photos of Michio Kaku's...,0,"[existing, photos, michio, kaku, betatron, ope..."
697138,88852a7e529aba265889,How do we evaluate people?,0,"[evaluate, people]"


In [15]:
train_df['p_txtcn'] = train_df['p_txt'].apply(lambda tokens: ' '.join(str(v) for v in tokens))
test_df['p_txtcn'] = test_df['p_txt'].apply(lambda tokens: ' '.join(str(v) for v in tokens))

In [37]:
no_features = 500
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=no_features)
# tf_vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=no_features)

In [38]:
def vectorize(txt, vectorizer):
    X = vectorizer.fit_transform(txt)
    feature_names = vectorizer.get_feature_names()
    return X, feature_names, vectorizer

In [45]:
# X_tf, tf_feature_names, vectorizer = vectorize(train_df['p_txtcn'], tfidf_vectorizer)
X_tf_lda, tf_feature_names_lda, vectorizer_lda = vectorize(train_df['p_txtcn'], tfidf_vectorizer)

In [51]:
no_topics = 10
num_iter = 2

In [52]:
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=num_iter, learning_method='online', learning_offset=50.,random_state=9, evaluate_every=-1).fit(X_tf_lda)

In [53]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        top_features = []
        for i in topic.argsort()[:-no_top_words - 1:-1]:
            if topic[i] >  1.0:
                top_features.append((feature_names[i], topic[i]))
            else:
                continue        
        print("Topic %d:" % (topic_idx))
        print(", ".join([str(val[0])+": "+"%.2f" % val[1] for val in top_features]))

In [54]:
display_topics(lda, tf_feature_names_lda, 5)

Topic 0:
men: 3233.52, sex: 2905.78, liberals: 2379.40, make: 1965.18, support: 972.60
Topic 1:
americans: 2989.64, girls: 2856.86, get: 2680.32, gay: 1853.83, true: 1697.88
Topic 2:
think: 3567.83, black: 2795.41, trump: 2187.86, american: 1901.79, people: 1896.63
Topic 3:
india: 3162.79, chinese: 2299.92, much: 2060.40, feel: 1699.90, good: 1664.96
Topic 4:
women: 5626.34, would: 3016.58, us: 2587.84, jews: 1772.29, believe: 1558.40
Topic 5:
want: 2278.29, america: 1683.73, democrats: 1486.76, china: 1368.38, countries: 1327.58
Topic 6:
trump: 3358.30, white: 3279.00, indian: 3178.71, people: 1552.02, racist: 1407.94
Topic 7:
muslims: 3524.84, hate: 2679.74, muslim: 2056.69, one: 1771.60, world: 1481.78
Topic 8:
people: 4184.74, many: 3306.77, indians: 2878.93, country: 1908.54, really: 1613.29
Topic 9:
like: 3997.61, quora: 3592.62, president: 1569.47, questions: 1402.33, even: 1187.51


In [55]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda, X_tf_lda, vectorizer_lda, mds='tsne')
panel

/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
4     -117.727615 -132.594223       1        1  10.938934
2      113.927292   95.090324       2        1  10.268970
5      253.289703 -278.334167       3        1  10.175497
6     -258.650452  242.431931       4        1  10.149837
8     -375.552368 -389.463348       5        1   9.960029
1       -5.128246 -520.291626       6        1   9.931058
3      499.706390  -24.195478       7        1   9.875112
7        4.754779  480.681854       8        1   9.743104
9     -501.411987  -16.822897       9        1   9.505497
0      375.205811  348.394379      10        1   9.451962, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
484   Default  4493.000000       women  4493.000000  30.0000  30.0000
256   Default  4442.000000        like  4442.000000  29.0000  29.0000
304   Default  3269.000000     muslims  3269.000000  28.0000  28.0000
362   Default  3153.000000       quora  3153.000000  27.0000  27.0000
446   Default  4411.000000       trump  4411.000000  26.0000  26.0000
215   Default  3033.000000       india  3033.000000  25.0000  25.0000
337   Default  7624.000000      people  7624.000000  24.0000  24.0000
287   Default  3270.000000         men  3270.000000  23.0000  23.0000
276   Default  2673.000000        many  2673.000000  22.0000  22.0000
478   Default  2689.000000       white  2689.000000  21.0000  21.0000
438   Default  2702.000000       think  2702.000000  20.0000  20.0000
216   Default  2606.000000      indian  2606.000000  19.0000  19.0000
198   Default  2485.000000        hate  2485.000000  18.0000  18.0000
396   Default  2400.000000         sex  2400.000000  17.0000  17.0000
14    Default  2446.000000   americans  2446.000000  16.0000  16.0000
176   Default  2338.000000       girls  2338.000000  15.0000  15.0000
217   Default  2327.000000     indians  2327.000000  14.0000  14.0000
70    Default  2206.000000     chinese  2206.000000  13.0000  13.0000
490   Default  2806.000000       would  2806.000000  12.0000  12.0000
172   Default  2610.000000         get  2610.000000  11.0000  11.0000
45    Default  2117.000000       black  2117.000000  10.0000  10.0000
466   Default  2091.000000        want  2091.000000   9.0000   9.0000
253   Default  1965.000000    liberals  1965.000000   8.0000   8.0000
301   Default  1976.000000        much  1976.000000   7.0000   7.0000
458   Default  2067.000000          us  2067.000000   6.0000   6.0000
303   Default  1908.000000      muslim  1908.000000   5.0000   5.0000
271   Default  1623.000000        make  1623.000000   4.0000   4.0000
323   Default  1643.000000         one  1643.000000   3.0000   3.0000
150   Default  1630.000000        feel  1630.000000   2.0000   2.0000
182   Default  1597.000000        good  1597.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
271   Topic10  1623.009201        make  1623.777663   2.3585  -2.9370
424   Topic10   803.254872     support   804.023315   2.3580  -3.6404
119   Topic10   716.092822         eat   716.861204   2.3579  -3.7552
402   Topic10   687.717712      sister   688.486083   2.3578  -3.7957
34    Topic10   665.378936        back   666.147383   2.3578  -3.8287
136   Topic10   655.944093    everyone   656.712524   2.3578  -3.8430
294   Topic10   649.707955         mom   650.476322   2.3578  -3.8525
363   Topic10   615.620504        race   616.388930   2.3577  -3.9064
4     Topic10   607.059849    actually   607.828282   2.3577  -3.9204
274   Topic10   598.333993        male   599.102381   2.3577  -3.9349
345   Topic10   559.634063        poor   560.402491   2.3576  -4.0017
152   Topic10   555.318120   feminists   556.086501   2.3576  -4.0095
151   Topic10   553.463558      female   554.231943   2.3576  -4.0128
101   Topic10   546.446723         day   

In [30]:
X_lda = lda.transform(X_tf_lda)
y_clf = train_df['target']
X_lda_train, X_lda_test, y_lda_train, y_lda_test = train_test_split(X_lda, y_clf, test_size=0.2, random_state=0, stratify=y_clf)
# X_train, X_test, y_train, y_test = train_test_split(X_tf, y_clf, test_size=0.2, random_state=0, stratify=y_clf)

In [ ]:
# y_svm = train_df['target']

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler().fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

In [31]:
# from sklearn.svm import SVC
# clf = SVC(C=10, kernel='linear', random_state=9)
# clf = RandomForestClassifier(n_estimators=100, random_state=9)
# clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')

# clf.fit(X_train, y_train)

clf2 = RandomForestClassifier(n_estimators=100, random_state=9)
clf2.fit(X_lda_train, y_lda_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=9, verbose=0, warm_start=False)

In [32]:
# p_pred_y = clf.predict(X_test)
# confusion_matrix(y_test, p_pred_y)

p_pred_y_lda = clf2.predict(X_lda_test)
confusion_matrix(y_lda_test, p_pred_y_lda)

array([[118735,   3779],
       [ 13335,  10926]])

In [33]:
def print_cm(y_test, p_pred_y):
    tn, fp, fn, tp = confusion_matrix(y_test, p_pred_y).ravel()

    acc = (tp + tn) / (tn + fp + fn + tp)
    prec = tp / (tp + fp)
    rec = tp / (tp + fn)
    f1 = 2 * prec * rec / (prec + rec)

    print("accuracy", acc)
    print("precision", prec)
    print("recall", rec)
    print("f1", f1)
    print(tn, fp, fn, tp)

In [34]:
# print_cm(y_test, p_pred_y)
print_cm(y_lda_test, p_pred_y_lda)

# print_cm(p_pred_y, p_pred_y_lda)

accuracy 0.8833997615397717
precision 0.7430125807548453
recall 0.45035241746012117
f1 0.5607965919006312
118735 3779 13335 10926


In [ ]:
X_init_lda_test = lda.transform(vectorizer_lda.transform(test_df['p_txtcn']))
# X_init_lda_test = scaler.transform(X_init_lda_test)

In [ ]:
y_pred_final = clf.predict(vectorizer.transform(test_df['p_txtcn']))
y_pred_final_lda = clf2.predict(X_init_lda_test)

y_1 = np.maximum(y_pred_final, y_pred_final_lda)

In [ ]:
sub = pd.DataFrame({"qid":test_df["qid"].values})
sub['prediction'] = y_1
sub.to_csv("submission.csv", index=False)

In [ ]:
# X_lda[:30]

In [ ]:
# X_lda_df = pd.DataFrame(X_lda)
# X_lda_df["idxmax"] = X_lda_df.idxmax(axis=1)
# X_lda_df.head(2)

In [ ]:
# y_true = train_df["target"]
# y_lda = X_lda_df["idxmax"]

In [ ]:
# print(confusion_matrix(y_true, y_lda)[:2])

#### Trying Deep Learning with GloVe

In [ ]:
def fit_get_sequences(df, vocab_size):
    tokenizer = Tokenizer(num_words=vocab_size)
    tokenizer.fit_on_texts(df['p_txt'])
    sequences = tokenizer.texts_to_sequences(df['p_txt'])
    return sequences, tokenizer, vocab_size

In [ ]:
def get_sequences(df, tokenizer):
    return tokenizer.texts_to_sequences(df['p_txt'])

In [ ]:
# vocab_size = 1000
# sequences, tokenizer, vocab_size = fit_get_sequences(train_df, vocab_size)
# word_index = tokenizer.word_index
# print('Found %s unique tokens.' % len(word_index))
# avg = sum(map(len, sequences)) / len(sequences)
# std = np.sqrt(sum(map(lambda x: (len(x) - avg) ** 2, sequences)) / len(sequences))
# print("Tokens avg {} and std {}".format(avg, std))

# max_length = 100
# X = pad_sequences(sequences, maxlen=max_length)
# X_init_test = pad_sequences(get_sequences(test_df, tokenizer), maxlen=max_length)

# y = to_categorical(np.asarray(train_df['target']))
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [ ]:
# y = to_categorical(np.asarray(train_df['target']))

In [ ]:
# print('Shape of data:', X.shape)
# print('Shape of labels:', y.shape)

In [ ]:
def generate_embeddings(vocab_size, word_index):
    EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
    embeddings_index = {}
    f = open(EMBEDDING_FILE)
    # In the dataset, each line represents a new word embedding
    # The line starts with the word and the embedding values follow
#     for line in f:
#         values = line.split()
#         word = values[0]
#         embedding = np.asarray(values[1:], dtype='float32')
#         embeddings_index[word] = embedding 
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in f if o.split(" ")[0] in word_index)
    f.close()

    all_embs = np.stack(embeddings_index.values())
    emb_mean = all_embs.mean()
    emb_std = all_embs.std()
    print(emb_mean, emb_std)
    embedding_dim = 300
    nb_words = min(vocab_size, len(word_index))  # How many words are there actually
    # Create a random matrix with the same mean and std as the embeddings
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embedding_dim))
    # The vectors need to be in the same position as their index.
    # Meaning a word with token 1 needs to be in the second row (rows start with zero) and so on
    # Loop over all words in the word index
    for word, i in word_index.items():
        # If we are above the amount of words we want to use we do nothing
        if i >= vocab_size:
            break
        # Get the embedding vector for the word
        embedding_vector = embeddings_index.get(word)
        # If there is an embedding vector, put it in the embedding matrix
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_dim, embedding_matrix    

In [ ]:
# embedding_dim, embedding_matrix = generate_embeddings(vocab_size, word_index)

In [ ]:
def generate_model(embedding_dim, embedding_matrix, max_length, vocab_size, ifembed=True):
    model = Sequential()
    # model.add(Embedding(vocab_size, 100, input_length=max_length))
    if ifembed:
        model.add(
            Embedding(vocab_size, embedding_dim, input_length=max_length, weights=[embedding_matrix], trainable=False))
        model.add(LSTM(32, return_sequences=True))
    else:
        model.add(LSTM(32, return_sequences=True, input_shape=(max_length)))
    model.add(Dropout(0.2))
    model.add(LSTM(32, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    model.summary()
    model.compile(loss='binary_crossentropy', optimizer='adam',
                  metrics=[metrics.mae, metrics.categorical_accuracy])
    return model

In [ ]:
# model = generate_model(embedding_dim, embedding_matrix, max_length, vocab_size)

In [ ]:
# batch_size = 1000
# model.fit(X_train, y_train, batch_size=batch_size, epochs=10, validation_split=0.1)

In [ ]:
# y_svm = train_df['target']
# X_train, X_test, y_train, y_test = train_test_split(X, y_svm, test_size=0.2, random_state=0, stratify=y)

# from sklearn.svm import SVC
# clf = SVC(C=100, kernel='linear', random_state=9)
# clf.fit(X_train, y_train)
# p_pred_y = clf.predict(X_test)

# confusion_matrix(y_test, p_pred_y)

In [ ]:
def c_matrix(y_true, y_pred, num_classes=2):
    cm = confusion_matrix(y_true.argmax(axis=1), y_pred.argmax(axis=1))
    print(cm)
    cm_np = np.asarray(cm)
    TP = np.diag(cm_np)
    FP = np.sum(cm, axis=0) - TP
    FN = np.sum(cm, axis=1) - TP
    TN = []
    for i in range(num_classes):
        temp = np.delete(cm, i, 0)
        temp = np.delete(temp, i, 1)
        TN.append(sum(sum(temp)))
    prec = TP / (TP + FP)
    rec = TP / (TP + FN)
    acc = (TP + TN) / (TP + FP + TN + FN)
    f1 = 2 * prec * rec / (prec + rec)
    print("precision", prec)
    print("recall", rec)
    print("accuracy", acc)
    print("f1", f1)
    return prec, rec, acc, f1

In [ ]:
# y_pred = model.predict(X_test)

# c_matrix(y_test, y_pred, num_classes=2)

In [ ]:
# y_pred_final = model.predict(X_init_test)

In [ ]:
# y_1 = y_pred_final.argmax(axis=1)

In [ ]:
# sub = pd.DataFrame({"qid":test_df["qid"].values})
# sub['prediction'] = y_1
# sub.to_csv("submission.csv", index=False)